# TryAlgo in Paris

In [9]:
from paris import read_graph, closest_node, display

N, paris_graph, weight, distance, visited, paris_coords = read_graph()

## Géolocalisation du 24 rue Béranger

In [15]:
from geopy.geocoders import Nominatim

geocoder = Nominatim()
start = geocoder.geocode("24 rue Béranger, Paris")
start.longitude, start.latitude

(2.3634347, 48.8666687)

In [26]:
node = closest_node(paris_coords, start)
node

4355

In [35]:
paris_graph[node]

[8872, 11054]

## Balades gloutonnes

In [38]:
def take_first(start_node, length):
    path = [start_node]
    node = start_node
    for _ in range(length):
        node = paris_graph[node][0]
        path.append(node)
    return path

path = take_first(node, 10)
path

[4355, 8872, 10862, 3224, 10862, 3224, 10862, 3224, 10862, 3224, 10862]

Ça boucle tout de même beaucoup.

In [37]:
display(paris_coords, path)

Essayez de prendre l'élément suivant, ou un voisin au hasard.

In [39]:
from tryalgo.dijkstra import dijkstra

def paris_cross(start_node, f):
    N, paris_graph, weight, distance, visited, paris_coords = read_graph()
    node = start_node
    path = []
    elapsed_time = 0
    score = 0
    while elapsed_time < 3600:  # While there is still time
        path.append(node)
        new_neighbors = list(filter(lambda neighbor: not visited[node][neighbor], paris_graph[node]))
        if new_neighbors:
            new_neighbors.sort(key=lambda neighbor: f(path, neighbor))
            chosen_neighbor = new_neighbors[-1]
            elapsed_time += weight[node][chosen_neighbor]
            score += distance[node][chosen_neighbor]
            visited[node][chosen_neighbor] = True
            visited[chosen_neighbor][node] = True
        else:
            stuck_node = node
            dist, prec = dijkstra(paris_graph, weight, stuck_node)
            for jump_time, node in sorted((dist[u], u) for u in range(N)):
                if any(not visited[node][neighbor] for neighbor in paris_graph[node]):
                    chosen_neighbor = node
                    elapsed_time += jump_time
                    break
            extra_path = []
            node = prec[chosen_neighbor]
            while node != stuck_node:
                extra_path.append(node)
                node = prec[node]
            path.extend(extra_path)
        node = chosen_neighbor
    print('Score: %d in %d seconds' % (score, elapsed_time))
    return path

## Première balade : marche aléatoire

In [40]:
from random import randint

def random_walk(path, neighbor):
    return randint(1, 100)

path = paris_cross(numa, random_walk)

Score: 27094 in 3608 seconds


In [41]:
display(paris_coords, path)

## Deuxième balade : la route la plus longue

In [42]:
def longest_road(path, neighbor):
    return distance[path[-1]][neighbor]

path = paris_cross(numa, longest_road)

Score: 25712 in 3607 seconds


In [43]:
display(paris_coords, path)

## Troisième balade : la route ayant le plus grand rapport longueur / temps

In [44]:
def best_road(path, neighbor):
    return distance[path[-1]][neighbor] / weight[path[-1]][neighbor]

path = paris_cross(numa, best_road)

Score: 27383 in 3605 seconds


In [45]:
display(paris_coords, path)